In [1]:
# import libraries
import pandas as pd
import datetime
from matplotlib import pyplot as plt
import numpy as np
import glob
import spacepy.datamodel as dm
import pandas as pd

import os
import pandas as pd
import datetime
from matplotlib import pyplot as plt
import numpy as np
import glob
import datetime
from scipy.signal import argrelextrema
from scipy.signal import find_peaks
import math
from tqdm import tqdm
from scipy import interpolate
import matplotlib.dates as mdates
from spacepy import pycdf
from scipy.optimize import curve_fit

from pathlib import Path

from astropy.time import Time
import cdflib


In [199]:
datetime.datetime(2018,5,29)

datetime.datetime(2018, 5, 29, 0, 0)

In [ ]:
# ARG
radar_list = glob.glob('../data/external/Madrigal/2018-2021/*/*.hdf5')
radar_list.sort()

fileswithne = []
for radar in radar_list:
    
    # metadata
    metaA1 = pd.DataFrame(dm.fromHDF5(radar)['Metadata']['Experiment Parameters'])
    metaA2 = np.array([x.decode() for x in metaA1['name']])
    metaA3 = np.array([x.decode() for x in metaA1['value']])

    metaA4 = pd.DataFrame({'name': metaA2,
              'value': metaA3})
    metaA4.to_csv('../data/processed/Madrigal/{filename}_metadata.csv'.format(filename =Path(radar_list[0]).name))

    # variables
    varA1 = pd.DataFrame(dm.fromHDF5(radar)['Metadata']['Data Parameters'])
    varA2 = np.array([x.decode() for x in varA1['mnemonic']])
    varA3 = np.array([x.decode() for x in varA1['description']])
    varA4 = np.array([x.decode() for x in varA1['units']])

    varA5 = pd.DataFrame({'mnemonic': varA2,
                  'description': varA3,
                      'units': varA4})
    varA5.to_csv('../data/processed/Madrigal/{filename}_variables.csv'.format(filename =Path(radar_list[0]).name))

    # append a table with files that have Ne     
    if len(varA5[(varA5['mnemonic'] == "NE")|(varA5['mnemonic'] == "NEL")] ) > 0:
        fileswithne.append(radar)

In [172]:
radar_list = glob.glob('../data/external/Madrigal/2018-2021/*/*.hdf5')
radar_list.sort()

len(radar_list)

7623

In [ ]:
len(fileswithne)

In [ ]:
fileswithne

In [ ]:
# check dates
gf_list = glob.glob('../data/interim/GRACEFO/KBRNE_relative_v1/dat/*')
gf_list.sort()

headerlist=['CDF Epoch', 'GPS', 'Latitude', 'Longitude', 'Radius', 'Latitude_QD', 'Longitude_QD','MLT GRACE_1_Position', 'GRACE_2_Position', 'Iono_Corr', 'Distance', 'Relative_Hor_TEC', 'Relative_Ne']

for gf_file in gf_list:
    gf = pd.read_csv(gf_file, sep='\s+', header=0,index_col=False, names=headerlist)
    print(gf)



In [ ]:
Time(gf['CDF Epoch'].values[0], format='cdf_epoch').datetime


In [174]:
# open aux files
iri_calib = pd.read_csv('../data/interim/GRACEFO/KBRNE_relative_v1/aux_files/GRACE-FO-IRI_CALIB.AUX',  skiprows=[0], sep='\s+',names = ['cdf-epoch', 'Mean-offset-IRI'])
outliers = pd.read_csv('../data/interim/GRACEFO/KBRNE_relative_v1/aux_files/GRACE-FO-OUTLIERS.AUX', skiprows=[0,1,2], sep='\s+')
gap_bias = pd.read_csv('../data/interim/GRACEFO/KBRNE_relative_v1/aux_files/GRACE-GAP-BIAS.ARC', sep='\s+')

## lear how to use the aux files ...

In [175]:
iri_calib

,cdf-epoch,Mean-offset-IRI
0,63694811097000,-7.840156e+10
1,63695601492000,-3.282334e+10
2,63696271092000,-1.734304e+10
3,63696275197000,-7.080232e+10
4,63696917847000,3.499636e+11
...,...,...
278,63776024302000,-1.603204e+11
279,63776109362000,-1.510191e+11
280,63776183027000,-1.276199e+11
281,63776194282000,-1.959966e+11


In [176]:
outliers

,cdf-epoch-start,cdf-epoch-end,Type
0,63696914547000,63696914742000,e
1,63696914742000,63696917722000,c
2,63696917722000,63696917847000,c
3,63696917847000,63696917947000,s
4,63737595992000,63737596092000,s
5,63746261332000,63746270257000,c
6,63765264742000,63765265492000,e
7,63765287602000,63765288537000,e
8,63768467602000,63768468422000,e
9,63769828942000,63769829482000,e


In [177]:
gap_bias

,YYYY,DDD,GPSS-GRACE,delta_ARC0,CDF_Epoch,new_ARC
0,2018,149,580863915,392872.083520,63694811097000,0
1,2018,149,580863915,392872.083520,63694811097000,1
2,2018,150,580910400,392872.083467,63694857582000,0
3,2018,151,580996800,392872.083413,63694943982000,0
4,2018,152,581083200,392872.083371,63695030382000,0
...,...,...,...,...,...,...
1111,2020,363,662441000,340584.078476,63776388182000,1
1112,2020,363,662446440,346468.357566,63776393622000,1
1113,2020,364,662472000,346468.357561,63776419182000,0
1114,2020,365,662558400,346468.357548,63776505582000,0


In [ ]:
# new arc flag 1

plt.plot(gap_bias['CDF_Epoch'][gap_bias['new_ARC']==1], gap_bias['new_ARC'][gap_bias['new_ARC']==1])

In [ ]:
len(np.unique(gap_bias['CDF_Epoch'][gap_bias['new_ARC']==1]))

In [ ]:
gap_bias[gap_bias['CDF_Epoch'] == 63694811097000]

In [ ]:
# plot radars

In [ ]:
# find conjunctions

In [3]:
# check dates
gf_list = glob.glob('../data/interim/GRACEFO/KBRNE_relative_v1/dat/*')
gf_list.sort()

headerlist=['CDF Epoch', 'GPS', 'Latitude', 'Longitude', 'Radius', 'Latitude_QD', 'Longitude_QD','MLT GRACE_1_Position', 'GRACE_2_Position', 'Iono_Corr', 'Distance', 'Relative_Hor_TEC', 'Relative_Ne']

grf_list_datetimes =[]
for gf_file in gf_list:
    gf = pd.read_csv(gf_file, sep='\s+', header=0,index_col=False, names=headerlist)
    grf_list_datetimes.append((Time(gf['CDF Epoch'].values[0], format='cdf_epoch').datetime).strftime('%Y-%m-%d'))

In [370]:
gf

,CDF Epoch,GPS,Latitude,Longitude,Radius,Latitude_QD,Longitude_QD,MLT GRACE_1_Position,GRACE_2_Position,Iono_Corr,Distance,Relative_Hor_TEC,Relative_Ne
0,63747647982000,1264464000,-72.807,-149.928,6892715.06,-68.212,-34.941,16.39,-1685131.2,-981916.9,-6611750.3,-1841145.7,-1059882.0
1,63747647987000,1264464005,-73.122,-149.885,6892721.64,-68.430,-34.306,16.44,-1652906.0,-965044.0,-6622366.6,-1809203.6,-1043097.8
2,63747647992000,1264464010,-73.437,-149.839,6892727.74,-68.646,-33.659,16.48,-1620618.7,-948165.5,-6632781.8,-1777194.7,-1026305.4
3,63747647997000,1264464015,-73.752,-149.791,6892733.37,-68.858,-33.000,16.53,-1588270.3,-931281.9,-6642995.8,-1745120.0,-1009505.5
4,63747648002000,1264464020,-74.066,-149.740,6892738.52,-69.068,-32.329,16.57,-1555861.7,-914393.9,-6653008.2,-1712980.4,-992698.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17275,63747734357000,1264550375,-26.683,25.314,6879564.96,-35.864,94.342,1.01,5592871.3,2647207.6,-3007683.0,5520476.8,2609330.0
17276,63747734362000,1264550380,-26.367,25.300,6879465.74,-35.600,94.449,1.01,5608385.4,2652865.4,-2973384.2,5536795.0,2615401.6
17277,63747734367000,1264550385,-26.050,25.286,6879366.22,-35.335,94.554,1.02,5623732.7,2658431.1,-2938994.1,5552949.0,2621381.6
17278,63747734372000,1264550390,-25.733,25.272,6879266.41,-35.069,94.658,1.03,5638912.6,2663904.6,-2904513.8,5568938.0,2627270.1


In [277]:
t1 = t.to_pandas()
t1.columns

Index(['year', 'month', 'day', 'hour', 'min', 'sec', 'recno', 'kindat',
       'kinst', 'ut1_unix', 'ut2_unix', 'az1', 'az2', 'el1', 'el2', 'hmax',
       'nemax', 'range', 'ne', 'dne', 'te', 'dte', 'ti', 'dti', 'vo', 'dvo',
       'ph+', 'dph+', 'phe+', 'dphe+', 'pm', 'dpm'],
      dtype='object')

In [301]:
radar = (Table.read(radar_list[i], path='Data/Table Layout')).to_pandas()

radar


,year,month,day,hour,min,sec,recno,kindat,kinst,ut1_unix,ut2_unix,azm,elm,hsa,power,systmp,range,pop,dpop,rgate
0,2018,1,26,8,2,20,0,6300,95,1.516954e+09,1.516954e+09,186.800003,81.599998,1.570796,721.013687,83.509857,85.154750,1.086642e+11,3.312882e+09,119.842852
1,2018,1,26,8,2,20,0,6300,95,1.516954e+09,1.516954e+09,186.800003,81.599998,1.570796,721.013687,83.509857,89.787844,2.982857e+10,1.446140e+09,119.842852
2,2018,1,26,8,2,20,0,6300,95,1.516954e+09,1.516954e+09,186.800003,81.599998,1.570796,721.013687,83.509857,95.695078,1.840239e+10,1.265071e+09,119.842852
3,2018,1,26,8,2,20,0,6300,95,1.516954e+09,1.516954e+09,186.800003,81.599998,1.570796,721.013687,83.509857,101.030055,1.812061e+10,2.052387e+09,119.842852
4,2018,1,26,8,2,20,0,6300,95,1.516954e+09,1.516954e+09,186.800003,81.599998,1.570796,721.013687,83.509857,104.777461,1.911856e+10,1.644040e+09,119.842852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37384,2018,1,26,19,59,48,362,6300,95,1.516997e+09,1.516997e+09,186.800003,81.599998,1.570796,667.299313,86.149078,599.435000,5.364900e+09,2.602722e+09,119.842852
37385,2018,1,26,19,59,48,362,6300,95,1.516997e+09,1.516997e+09,186.800003,81.599998,1.570796,667.299313,86.149078,604.717313,-7.832319e+09,2.884591e+09,119.842852
37386,2018,1,26,19,59,48,362,6300,95,1.516997e+09,1.516997e+09,186.800003,81.599998,1.570796,667.299313,86.149078,610.677250,-6.085314e+09,5.470015e+09,119.842852
37387,2018,1,26,19,59,48,362,6300,95,1.516997e+09,1.516997e+09,186.800003,81.599998,1.570796,667.299313,86.149078,614.424625,-3.054305e+06,2.585303e+09,119.842852


In [310]:
month = radar.month[0]

In [337]:
teste = ['gyaga']

In [338]:
teste

['gyaga']

In [339]:
len(teste)

1

In [320]:
import datetime
datetime.datetime(radar.year[0],radar.month[0],radar.day[0]).strftime('%Y-%m-%d')
# (().strftime('%Y-%m-%d'))

'2018-01-26'

In [330]:
radar_list[5000]

'../data/external/Madrigal/2018-2021/MillstoneHillISRadar/mlh180827g.003.hdf5'

In [361]:
i=0
radar_metadata = (Table.read(radar_list[i], path='Metadata/Experiment Parameters')).to_pandas()

names = np.array([x.decode() for x in radar_metadata['name']])
values = np.array([x.decode() for x in radar_metadata['value']])

radar_metadata = pd.DataFrame({'name': names,
                  'value': values})

In [367]:
radar_metadata[radar_metadata['name'] == 'instrument latitude']['value']

8    18.345
Name: value, dtype: object

In [378]:
gf_file = gf_list[9]
gf = pd.read_csv(grf_file, sep='\s+', header=0,index_col=False, names=headerlist)
gf['dates'] = (Time(gf['CDF Epoch'].values, format='cdf_epoch')).datetime

In [379]:
gf.columns

Index(['CDF Epoch', 'GPS', 'Latitude', 'Longitude', 'Radius', 'Latitude_QD',
       'Longitude_QD', 'MLT GRACE_1_Position', 'GRACE_2_Position', 'Iono_Corr',
       'Distance', 'Relative_Hor_TEC', 'Relative_Ne', 'dates'],
      dtype='object')

In [ ]:
i = 17





In [38]:
# parameters

spatial_window = 5 # km
temporal_window = 15 # min

GRf_freq = 5 # sec

MLH_lat = 42.61 # degrees
MLH_lon = -71.49 # degrees
MLH_timeUTC = -5 # hours


In [198]:
 
# for i in tqdm(range(0,len(radar_list))):
for i in tqdm(range(0,500)):
    print(i)

    metaA1 = pd.DataFrame(dm.fromHDF5(radar_list[i])['Metadata']['Experiment Parameters'])
    metaA2 = np.array([x.decode() for x in metaA1['name']])
    metaA3 = np.array([x.decode() for x in metaA1['value']])

    metaA4 = pd.DataFrame({'name': metaA2,
                  'value': metaA3})
    radar_date = pd.to_datetime(metaA4[metaA4['name'] == 'start time'].value).dt.strftime('%Y-%m-%d')    
    try:
        # get intersection date
        grf_idx = np.intersect1d(grf_list_datetimes,radar_date, return_indices=True)[1][0]-1

        # open files and add datetime as index
        ## MLH
        radar_file = radar_list[i] 
        radar = dm.fromHDF5(radar_file)
        radar_datetimes =[]
        for t in iter(range(0,len(radar['Data']['Array Layout']['timestamps']))):

            radar_datetimes.append(datetime.datetime.fromtimestamp(radar['Data']['Array Layout']['timestamps'][t])) 
        
        
        radar_df = pd.DataFrame({
                'dates': radar_datetimes,
        #         'ne' : (radar['Data']['Array Layout']['2D Parameters']['ne']).flatten(),
                'range': radar['Data']['Array Layout']['range'][0]
                })

        radar_df = radar_df.loc[radar_df.index.repeat(np.shape(radar['Data']['Array Layout']['2D Parameters']['ne'])[0])]

        radar_df['ne'] = (radar['Data']['Array Layout']['2D Parameters']['ne']).flatten()

        ## GRACE
        grf_file = gf_list[grf_idx]
        grf = pd.read_csv(grf_file, sep='\s+', header=0,index_col=False, names=headerlist)
        grf['dates'] = (Time(grf['CDF Epoch'].values, format='cdf_epoch')).datetime
        
        
        # find GRACE observations that are within spatial window
        MLH_lat = float(metaA4[metaA4['name'] == 'instrument latitude']['value'])
        MLH_lon = float(metaA4[metaA4['name'] == 'instrument longitude']['value'])
        
        grf = grf[((grf['Latitude'] <= MLH_lat + spatial_window) & (grf['Latitude'] >= MLH_lat - spatial_window))]
        grf = grf[((grf['Longitude'] <= MLH_lon + spatial_window) & (grf['Longitude'] >= MLH_lon - spatial_window))] 
        
        print(grf)

        if len(grf)>0:
#             print(radar_file)
#             print(grf_file)
            print('---------------------')
#             grf_pass = grf[grf.diff(axis = 0)['dates'] > datetime.timedelta(minutes=temporal_window)]
# #             print(len(grf_pass))
        
#             if len(grf_pass) == 0: # only one passage

# #                 print('GRF passage ', grf['dates'][grf.index.min()], '->', grf['dates'][grf.index.max()])
# #                 print(i)

#                 # define time window
#                 time_window_i = grf['dates'][grf.index.min()] - datetime.timedelta(minutes=temporal_window)
#                 time_window_f = grf['dates'][grf.index.max()] + datetime.timedelta(minutes=temporal_window)

# #                 filter values
#                 grf_select = grf[(grf['dates'] >= grf['dates'][grf.index.min()]) & (grf['dates'] <= grf['dates'][grf.index.max()])] # within time window
#                 mlh_select = radar_df[(radar_df['dates'] >= time_window_i) & (radar_df['dates'] <= time_window_f)]
#                 print(i, len(grf_select), len(mlh_select))
# #                 print(radar_file)
# #                 print(grf_file)
# #                 print('---------------------')
#                 if ((len(mlh_select) > 0) and (len(grf_select) > 0)):
#                                               print('olha eu aqui!!!!!!!!')
                    
# # #                         grf_select.to_csv('grf_{date}_v{version}_exp{exp}.csv'.format(date = mlh_date.strftime('%Y-%m-%d'),
# # #                                                                           version = mlh_file[38:41],
# # #                                                                           exp = mlh_file[42:50]))
# # #                         mlh_select.to_csv('mlh_{date}_v{version}_exp{exp}.csv'.format(date = mlh_date.strftime('%Y-%m-%d'),
# # #                                                                           version = mlh_file[38:41],
# # #                                                                           exp = mlh_file[42:50]))
# # #                 else:
# # #                     pass


# # #             if len(grf_pass) == 1: # two passages

# # #                 pass_idx = np.argwhere(grf.index == grf_pass.index[0])[0][0]

# # #                 # first one
# # #                 print('GRF passage', grf.index[0], '->', grf.index[pass_idx-1])

# # #                 # define time window
# # #                 time_window_i = grf.datetimes[grf.index[0]] - datetime.timedelta(minutes=15)
# # #                 time_window_f = grf.datetimes[grf.index[pass_idx-1]] + datetime.timedelta(minutes=15)

# # #                 # filter values
# # #                 grf_select = grf[(grf.index >= grf.index[0]) & (grf.index <= grf.index[pass_idx-1])] # within time window

# # #                 mlh_select = mlh[(mlh.index >= time_window_i) & (mlh.index <= time_window_f)]

# # # #                 if ((len(mlh_select) > 0) and (len(grf_select) > 0)):

# # # #                         grf_select.to_csv('grf_{date}_v{version}_exp{exp}.csv'.format(date = mlh_date.strftime('%Y-%m-%d'),
# # # #                                                                           version = mlh_file[38:41],
# # # #                                                                           exp = mlh_file[42:50]))
# # # #                         mlh_select.to_csv('mlh_{date}_v{version}_exp{exp}.csv'.format(date = mlh_date.strftime('%Y-%m-%d'),
# # # #                                                                           version = mlh_file[38:41],
# # # #                                                                           exp = mlh_file[42:50]))
# # # #                 else:
# # # #                     pass

# # #                 # second one
# # #                 print('GRF passage', grf.index[pass_idx], '->', grf.index[-1])

# # #                 # define time window
# # #                 time_window_i = grf.datetimes[grf.index[pass_idx]] - datetime.timedelta(minutes=15)
# # #                 time_window_f = grf.datetimes[grf.index[-1]] + datetime.timedelta(minutes=15)

# # #                 # filter values
# # #                 grf_select = grf[(grf.index >= grf.index[pass_idx]) & (grf.index <= grf.index[-1])] # within time window

# # #                 mlh_select = mlh[(mlh.index >= time_window_i) & (mlh.index <= time_window_f)]

# # # #                 if ((len(mlh_select) > 0) and (len(grf_select) > 0)):

# # # #                         grf_select.to_csv('grf_{date}_v{version}_exp{exp}.csv'.format(date = mlh_date.strftime('%Y-%m-%d'),
# # # #                                                                           version = mlh_file[38:41],
# # # #                                                                           exp = mlh_file[42:50]))
# # # #                         mlh_select.to_csv('mlh_{date}_v{version}_exp{exp}.csv'.format(date = mlh_date.strftime('%Y-%m-%d'),
# # # #                                                                           version = mlh_file[38:41],
# # # #                                                                           exp = mlh_file[42:50]))
# # # #                 else:
# # # #                     pass
        else:
            pass
    except:
        pass

  0%|          | 0/500 [00:00<?, ?it/s]

0


  1%|          | 3/500 [00:02<05:45,  1.44it/s]

1
2
3
4


  2%|▏         | 8/500 [00:02<01:45,  4.66it/s]

...

493
494


100%|█████████▉| 498/500 [00:58<00:00,  7.03it/s]

495
496
497
498


100%|██████████| 500/500 [00:58<00:00,  8.51it/s]

Empty DataFrame
Columns: [CDF Epoch, GPS, Latitude, Longitude, Radius, Latitude_QD, Longitude_QD, MLT GRACE_1_Position, GRACE_2_Position, Iono_Corr, Distance, Relative_Hor_TEC, Relative_Ne, dates]
Index: []
499


In [180]:
radar_df = pd.DataFrame({
        'dates': radar_datetimes,
#         'ne' : (radar['Data']['Array Layout']['2D Parameters']['ne']).flatten(),
        'range': radar['Data']['Array Layout']['range'][0]
        })
        
radar_df = radar_df.loc[radar_df.index.repeat(85)]

radar_df['ne'] = (radar['Data']['Array Layout']['2D Parameters']['ne']).flatten()





KeyError: 'ne'

In [187]:
radar_df['dates'] <= time_window_i

0      True
0      True
0      True
0      True
0      True
       ... 
287    True
287    True
287    True
287    True
287    True
Name: dates, Length: 24480, dtype: bool

In [184]:
time_window_i

Timestamp('2019-10-25 07:28:22')

In [145]:
radar_file = radar_list[0] 
radar = dm.fromHDF5(radar_file)
radar_datetimes =[]
for t in iter(range(0,len(radar['Data']['Array Layout']['timestamps']))):

    radar_datetimes.append(datetime.datetime.fromtimestamp(radar['Data']['Array Layout']['timestamps'][t])) 
        

np.shape(radar['Data']['Array Layout']['2D Parameters']['ne'])[0]

85

In [136]:
np.shape(radar['Data']['Array Layout']['range'])[0]

85

In [106]:
grf['dates']

15021   2020-08-20 20:51:27
15022   2020-08-20 20:51:32
15023   2020-08-20 20:51:37
15024   2020-08-20 20:51:42
15025   2020-08-20 20:51:47
15026   2020-08-20 20:51:52
15027   2020-08-20 20:51:57
15028   2020-08-20 20:52:02
15029   2020-08-20 20:52:07
15030   2020-08-20 20:52:12
15031   2020-08-20 20:52:17
15032   2020-08-20 20:52:22
15033   2020-08-20 20:52:27
15034   2020-08-20 20:52:32
15035   2020-08-20 20:52:37
15036   2020-08-20 20:52:42
15037   2020-08-20 20:52:47
15038   2020-08-20 20:52:52
15039   2020-08-20 20:52:57
15040   2020-08-20 20:53:02
15041   2020-08-20 20:53:07
15042   2020-08-20 20:53:12
15043   2020-08-20 20:53:17
15044   2020-08-20 20:53:22
15045   2020-08-20 20:53:27
15046   2020-08-20 20:53:32
15047   2020-08-20 20:53:37
15048   2020-08-20 20:53:42
15049   2020-08-20 20:53:47
15050   2020-08-20 20:53:52
15051   2020-08-20 20:53:57
15052   2020-08-20 20:54:02
Name: dates, dtype: datetime64[ns]

In [126]:
# dm.fromHDF5(radar_list[0])['Data']['Array Layout']['2D Parameters']['ne']

# dm.fromHDF5(radar_list[0])['Data']['Array Layout']['range']

dmarray([[7.00000e+05, 6.33403e+10, 6.03664e+10, ..., 4.44310e+09,
          4.89110e+09, 4.32810e+09],
         [1.06050e+11, 1.09169e+11, 1.06083e+11, ..., 8.85060e+09,
          7.12180e+09, 3.94490e+09],
         [1.11636e+11, 1.19121e+11, 1.16032e+11, ..., 3.77630e+09,
          5.68000e+09, 2.80640e+09],
         ...,
         [3.41318e+10, 5.72159e+10, 4.95072e+10, ..., 1.15873e+10,
          1.27142e+10, 1.27567e+10],
         [3.41118e+10, 5.77888e+10, 4.98728e+10, ..., 1.15873e+10,
          1.27142e+10, 1.27567e+10],
         [3.40942e+10, 5.83566e+10, 5.02411e+10, ..., 1.15873e+10,
          1.27142e+10, 1.27567e+10]])

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
float(metaA4[metaA4['name'] == 'instrument latitude']['value']) + 5

72.4

In [24]:
gf_file = gf_list[grf_idx]
gf = pd.read_csv(gf_file, sep='\s+', header=0,index_col=False, names=headerlist)
# gf['dates'] = (Time(gf['CDF Epoch'].values, format='cdf_epoch')).datetime

#         #             gf_pass = gf[gf.diff(axis = 0)['dates'] > datetime.timedelta(minutes=30)]


In [26]:
gf.columns

Index(['CDF Epoch', 'GPS', 'Latitude', 'Longitude', 'Radius', 'Latitude_QD',
       'Longitude_QD', 'MLT GRACE_1_Position', 'GRACE_2_Position', 'Iono_Corr',
       'Distance', 'Relative_Hor_TEC', 'Relative_Ne'],
      dtype='object')